In [3]:
from pypokerengine.players import BasePokerPlayer

class FishPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [5]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

NB_SIMULATION = 1000

class HonestPlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=NB_SIMULATION,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        if win_rate >= 1.0 / self.nb_player:
            action = valid_actions[1]  # fetch CALL action info
        else:
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [ ]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

NB_SIMULATION = 1000

class RationalPlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=NB_SIMULATION,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        if win_rate >= 1.0 / self.nb_player:
            action = valid_actions[1]  # fetch CALL action info
        else:
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass
    
    def ev_calculation(win_rate, pot,self_bet, small_blind_amount):
        ev = 
        

In [17]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

NB_SIMULATION = 1000

class RationalPlayer(BasePokerPlayer):


    def declare_action(self, valid_actions, hole_card, round_state):
        community_card = round_state['community_card']
        call_amount = valid_actions[1]['amount']
        pot = round_state['pot']['main']['amount']
        win_rate = estimate_hole_card_win_rate(nb_simulation = NB_SIMULATION,
                                                nb_player = self.nb_player,
                                                hole_card=gen_cards(hole_card),
                                                community_card=gen_cards(community_card))
        ev = self.ev_calculation(win_rate, pot, call_amount)

        if ev.index(max(ev)) == 0:
            return valid_actions[0]['action'], valid_actions[0]['amount']

        elif ev.index(max(ev)) == 1:
            return valid_actions[1]['action'], valid_actions[1]['amount']

        else:
            return valid_actions[2]['action'], 2 * self.small_blind_amount
                        


    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']
        self.uuid = game_info['seats'][0]
        self.small_blind_amount = game_info['rule']['small_blind_amount']

    def receive_round_start_message(self, round_count, hole_card, seats):
        self.self_bet = 0

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        if action['player_uuid'] == self.uuid:
            self.self_bet = self.self_bet + action['amount']

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

    def ev_calculation(self, win_rate, pot, call_amount):
        ev = [0 for i in range(3)]
        ev[0] = -self.self_bet
        ev[1] = win_rate * (pot - self.self_bet) - (1 - win_rate) * (self.self_bet + call_amount)
        ev[2] = win_rate * (pot - self.self_bet + 2 * self.small_blind_amount) - (1 - win_rate) * (self.self_bet + 2 * self.small_blind_amount + call_amount)
        return ev

In [21]:
from pypokerengine.api.game import setup_config, start_poker

config = setup_config(max_round=100, initial_stack=1000, small_blind_amount=1)
config.register_player(name="p1", algorithm=WisePlayer(0.5, 0.5))
config.register_player(name="p2", algorithm=RationalPlayer())
game_result = start_poker(config, verbose=1)

Started the round 1
Street "preflop" started. (community card = [])
"p2" declared "fold:0"
"['p1']" won the round 1 (stack = {'p1': 1001, 'p2': 999})
Started the round 2
Street "preflop" started. (community card = [])
"p1" declared "raise:2"
"['p2']" won the round 2 (stack = {'p1': 1000, 'p2': 1000})
Started the round 3
Street "preflop" started. (community card = [])
"p2" declared "fold:0"
"['p1']" won the round 3 (stack = {'p1': 1001, 'p2': 999})
Started the round 4
Street "preflop" started. (community card = [])
"p1" declared "raise:2"
"['p2']" won the round 4 (stack = {'p1': 1000, 'p2': 1000})
Started the round 5
Street "preflop" started. (community card = [])
"p2" declared "call:2"
"p1" declared "raise:2"
"['p2']" won the round 5 (stack = {'p1': 998, 'p2': 1002})
Started the round 6
Street "preflop" started. (community card = [])
"p1" declared "raise:2"
"['p2']" won the round 6 (stack = {'p1': 997, 'p2': 1003})
Started the round 7
Street "preflop" started. (community card = [])
"p2

"p1" declared "call:0"
Street "river" started. (community card = ['S8', 'SA', 'DA', 'H9', 'H3'])
"p2" declared "call:0"
"p1" declared "call:0"
"['p2']" won the round 49 (stack = {'p1': 984, 'p2': 1016})
Started the round 50
Street "preflop" started. (community card = [])
"p1" declared "raise:2"
"['p2']" won the round 50 (stack = {'p1': 983, 'p2': 1017})
Started the round 51
Street "preflop" started. (community card = [])
"p2" declared "fold:0"
"['p1']" won the round 51 (stack = {'p1': 984, 'p2': 1016})
Started the round 52
Street "preflop" started. (community card = [])
"p1" declared "raise:2"
"['p2']" won the round 52 (stack = {'p1': 983, 'p2': 1017})
Started the round 53
Street "preflop" started. (community card = [])
"p2" declared "raise:2"
"['p1']" won the round 53 (stack = {'p1': 984, 'p2': 1016})
Started the round 54
Street "preflop" started. (community card = [])
"p1" declared "raise:2"
"['p2']" won the round 54 (stack = {'p1': 983, 'p2': 1017})
Started the round 55
Street "pref

"p2" declared "raise:2"
"['p1']" won the round 97 (stack = {'p1': 976, 'p2': 1024})
Started the round 98
Street "preflop" started. (community card = [])
"p1" declared "raise:2"
"['p2']" won the round 98 (stack = {'p1': 975, 'p2': 1025})
Started the round 99
Street "preflop" started. (community card = [])
"p2" declared "raise:2"
"['p1']" won the round 99 (stack = {'p1': 976, 'p2': 1024})
Started the round 100
Street "preflop" started. (community card = [])
"p1" declared "call:2"
"p2" declared "call:2"
Street "flop" started. (community card = ['S2', 'H9', 'C6'])
"p1" declared "call:0"
"p2" declared "call:0"
Street "turn" started. (community card = ['S2', 'H9', 'C6', 'C5'])
"p1" declared "raise:2"
"p2" declared "raise:2"
"['p1']" won the round 100 (stack = {'p1': 978, 'p2': 1022})


Because of the equalty between win rate and the combination hole cards and community card, It's not a bad idea directly make use of win rate to construct the program.

How to deal with the get taught and exploited problem -- now by opponent modeling

Modelling the opponent is eqaul to find the suboptimal solution. Because the Nash equlibrium does exist.

In which frequency must the strategy be updated? -- now each round

How to describe the flexibility of a player -- with KL-divergence



In [14]:
import sqlite3 as lite
import sys
 
con = lite.connect('user.db')

with con:
    
    cur = con.cursor()
#     cur.execute("CREATE TABLE Users(Id INT, Name TEXT)")
    cur.execute("INSERT INTO Users VALUES(1,'Michelle')")
    cur.execute("INSERT INTO Users VALUES(2,'Sonya')")

In [19]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
import random as rand

NB_SIMULATION = 1000

class WisePlayer(BasePokerPlayer):
    def __init__(self, raise_threshold, bluffing_ratio):
        self.raise_threshold = raise_threshold
        self.bluffing_ratio = bluffing_ratio

    def declare_action(self, valid_actions, hole_card, round_state):
        community_card = round_state['community_card']
        call_amount = valid_actions[1]['amount']
        pot = round_state['pot']['main']['amount']
        win_rate = estimate_hole_card_win_rate(nb_simulation = NB_SIMULATION,
                                                nb_player = self.nb_player,
                                                hole_card=gen_cards(hole_card),
                                                community_card=gen_cards(community_card))
        return self.__choose_action(win_rate, pot, valid_actions)

    def __choose_action(self, win_rate, pot, valid_actions):                    
        r = rand.random()
        ev = self.ev_calculation(win_rate, pot, valid_actions[1]['amount'])
        if win_rate >= self.raise_threshold:
            return valid_actions[2]['action'], 2 * self.small_blind_amount
        elif r >= self.bluffing_ratio:
            return valid_actions[2]['action'], 2 * self.small_blind_amount
        elif ev[1] >= ev[0]:
            return valid_actions[1]['action'], valid_actions[1]['amount']
        else:
            return valid_actions[0]['action'], valid_actions[0]['amount']
            


    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']
        self.uuid = game_info['seats'][0]
        self.small_blind_amount = game_info['rule']['small_blind_amount']

    def receive_round_start_message(self, round_count, hole_card, seats):
        self.self_bet = 0

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        if action['player_uuid'] == self.uuid:
            self.self_bet = self.self_bet + action['amount']

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

    def write_history(self, win_rate):
        pass

    def ev_calculation(self, win_rate, pot, call_amount):
        ev = [0 for i in range(3)]
        ev[0] = -self.self_bet
        ev[1] = win_rate * (pot - self.self_bet) - (1 - win_rate) * (self.self_bet + call_amount)
        ev[2] = win_rate * (pot - self.self_bet + 2 * self.small_blind_amount) - (1 - win_rate) * (self.self_bet + 2 * self.small_blind_amount + call_amount)
        return ev    